In [1]:
!pip install transformers datasets


from datasets import load_dataset

# Load the dataset
dataset = load_dataset('ag_news')
train_data = dataset['train'].select(range(2000))  # Select a subset for faster training
test_data = dataset['test'].select(range(500))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [2]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)
train_data = train_data.remove_columns(['text'])
test_data = test_data.remove_columns(['text'])
train_data.set_format('torch')
test_data.set_format('torch')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [3]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,    ####try the same thing in pytorch and see if its working
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
trainer.train()


Step,Training Loss
10,1.368500
20,1.370000
30,1.362700
40,1.356700
50,1.320400
60,1.309300
70,1.236800
80,1.179500
90,1.107200
100,0.966400


TrainOutput(global_step=375, training_loss=0.6258174549738567, metrics={'train_runtime': 3964.1368, 'train_samples_per_second': 1.514, 'train_steps_per_second': 0.095, 'total_flos': 198708185088000.0, 'train_loss': 0.6258174549738567, 'epoch': 3.0})

In [5]:
eval_result = trainer.evaluate()
print(f"Evaluation result: {eval_result}")


Evaluation result: {'eval_loss': 0.3727438449859619, 'eval_runtime': 99.4284, 'eval_samples_per_second': 5.029, 'eval_steps_per_second': 0.322, 'epoch': 3.0}


In [7]:
import torch

def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class
    predicted_class = torch.argmax(logits, dim=1).item()

    # Define the labels (same order as training)
    labels = ["World", "Sports", "Business", "Sci/Tech"]

    return labels[predicted_class]

# Example text to classify
text = "Wall Street bounces back as tech stocks surge. Major tech firms led a rally in the stock market today, pushing indices higher."

# Get the prediction
predicted_label = predict(text)

print(f"Predicted Label: {predicted_label}")

Predicted Label: Business
